In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException


5성급 호텔 리뷰 웹크롤링

In [104]:
# 크롬 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [25]:
# 아고다 접속
url = 'https://www.agoda.com/ko-kr/'
driver.get(url)

In [26]:
# 광고 제거
driver.find_element('xpath','/html/body/div[12]/div[2]/div[3]/button[2]').click()

In [27]:
# 검색창 클릭
driver.find_element('xpath','//*[@id="textInput"]').click()

In [28]:
# 서울 입력
driver.find_element('xpath','//*[@id="textInput"]').send_keys('서울')

In [29]:
# 서울 클릭
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/ul/li[1]').click()

In [30]:
# 날짜 자동입력

actions = ActionChains(driver)

actions.send_keys(Keys.SPACE).perform()
actions.send_keys(Keys.SPACE).perform()

In [31]:
# 검색하기
driver.find_element('xpath','//*[@id="SearchBoxContainer"]/div[2]/div/button').click()

In [32]:
# 탭 이동
driver.switch_to.window(driver.window_handles[0])

# 탭 닫기
driver.close()

# 탭 바꾸기
driver.switch_to.window(driver.window_handles[0])

In [33]:
# 호텔 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(2) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [34]:
# 5성급 체크
driver.find_element(By.CSS_SELECTOR,'#SideBarLocationFilters > div:nth-child(3) > div.filter-items > ul > li:nth-child(1) > span > span:nth-child(1) > span').click()

In [35]:
# 스크롤 다운
actions = ActionChains(driver)
last_height = driver.execute_script('return document.body.scrollHeight')
while True:
    for _ in range (15):
        actions.send_keys(Keys.SPACE).perform()
        time.sleep(1)
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == last_height:
        break
    last_height = new_height


In [36]:
# 호텔명 가져오기
names = []

name_elements = driver.find_elements(By.CSS_SELECTOR, '.sc-jrAGrp.sc-kEjbxe.eDlaBj.dscgss')

for element in name_elements:
    text = element.text
    names.append(text)

len(names)

38

In [37]:
#호텔 url 가져오기
urls = []

url_elements = driver.find_elements(By.CLASS_NAME, 'PropertyCard__Link')
for element in url_elements:
    url = element.get_attribute('href')
    urls.append(url)
len(urls)

38

In [38]:
df = pd.DataFrame({
    'name':names,
    'url':urls,
    'address':None,
    'rating':None,
    'review_rating':None,
    'review_star' : None,
    'review':None
})

In [550]:

# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [559]:
driver.find_element(By.CSS_SELECTOR,'#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select').click()

In [560]:
# 언어 선택 드롭다운 찾기
language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))

# "한국어" 선택
language_dropdown.select_by_visible_text("한국어")

In [93]:
# 주소, 별점, 리뷰 가져오기

address = []
scores = []
reviews = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(6)
    
    # 주소 가져오기
    try:
        address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    except ElementClickInterceptedException:
        continue
    
    # 이용후기 누르기
    try:
        driver.find_element(By.XPATH, '//*[@id="hotelNavBar"]/div/ul/li[4]/button/span').click()
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # 언어 변경
    try:
        language_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select'))
        # driver.find_element(By.XPATH, '//*[@id="reviewFilterSection"]/div[1]/div[3]/span/i[2]').click() #21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    # "한국어"를 선택
    try:
        language_dropdown.select_by_visible_text("한국어")
        # driver.find_element(By.XPATH, '//*[@id="reviews-language-filter_list"]/ul/li[2]').click() # 21번 전까지
        time.sleep(2)
    except ElementClickInterceptedException:
        continue
    
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break
    
    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
    
    # 다음 호텔로 넘어가기
    continue

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#reviewFilterSection > div.sc-bdfBwQ.sc-gsTCUz.gdcQLK > div.sc-bdfBwQ.sc-gsTCUz.djZOQg > div > div > label > div.sc-bdfBwQ.sc-gsTCUz.bqqCNI > span > select"}
  (Session info: chrome=120.0.6099.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x006D6EE3+174339]
	(No symbol) [0x00600A51]
	(No symbol) [0x00316FF6]
	(No symbol) [0x00349876]
	(No symbol) [0x00349C2C]
	(No symbol) [0x0037BD42]
	(No symbol) [0x00367054]
	(No symbol) [0x0037A104]
	(No symbol) [0x00366DA6]
	(No symbol) [0x00341034]
	(No symbol) [0x00341F8D]
	GetHandleVerifier [0x00774B1C+820540]
	sqlite3_dbdata_init [0x008353EE+653550]
	sqlite3_dbdata_init [0x00834E09+652041]
	sqlite3_dbdata_init [0x008297CC+605388]
	sqlite3_dbdata_init [0x00835D9B+656027]
	(No symbol) [0x0060FE6C]
	(N

In [ ]:
df

In [ ]:
# 데이터 따로 입력해주기

stars = []
address = []
scores = []
reviews = []

for i in range(1,2):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(10)
    
    # 주소
    address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
        
    while True:
        # 별점 가져오기
        star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
        for element in star_elements:
            score = element.text
            scores.append(score)
            
        # 리뷰 가져오기
        review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
        for element in review_elements:
            review = element.text
            reviews.append(review)
            
        # 페이지 넘기기
        try:
            driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
            time.sleep(2)
        except ElementClickInterceptedException:
            # 다음 페이지가 없으면 while 루프 종료
            break

    # 데이터 추가하기
    if len(address) == 1:
        df.iloc[i]['address'] = address
        address = []
     
    if len(scores) > 0:
        df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
        df.iloc[i]['review_star'] = scores
        scores = []
    
    if len(reviews) > 0:
        df.iloc[i]['review'] = reviews
        reviews = []
        
    # 다음 호텔로 넘어가기
    continue

In [103]:
# 호텔 url 들어가기 
driver.get(urls[1])
time.sleep(10)

In [ ]:
# 데이터 따로 입력해주기

stars = []
address = []
scores = []
reviews = []



    
# 주소
address.append(driver.find_element('xpath','//*[@id="property-main-content"]/div[1]/div[2]/div[2]/span[1]').text)
    
while True:
    # 별점 가져오기
    star_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-leftScore')
    for element in star_elements:
        score = element.text
        scores.append(score)
        
    # 리뷰 가져오기
    review_elements = driver.find_elements(By.CLASS_NAME,'Review-comment-bodyText')
    for element in review_elements:
        review = element.text
        reviews.append(review)
        
    # 페이지 넘기기
    try:
        driver.find_element('xpath','//*[@id="reviewSection"]/div[4]/div/span[3]/i').click()
        time.sleep(2)
    except ElementClickInterceptedException:
        # 다음 페이지가 없으면 while 루프 종료
        break

# 데이터 추가하기
if len(address) == 1:
    df.iloc[i]['address'] = address
    address = []
    
if len(scores) > 0:
    df.iloc[i]['review_rating'] = int(sum(map(float,scores))) / len(scores)
    df.iloc[i]['review_star'] = scores
    scores = []

if len(reviews) > 0:
    df.iloc[i]['review'] = reviews
    reviews = []
    
# 다음 호텔로 넘어가기
continue

In [72]:
stars = []

for i in range(len(urls)):
    # 호텔 url 들어가기 
    driver.get(urls[i])
    time.sleep(2)
    
    # 홈페이지 총점 가져오기
    try :
        stars.append(float(driver.find_element('xpath','//*[@id="property-critical-root"]/div/div[5]/div[2]/div[1]/div[1]/div/div[1]/div/div/div/div/div/div/p[1]').text))
    except NoSuchElementException:
        stars.append('')

# 데이터 추가하기
df['rating'] = stars

In [622]:
df.to_csv('./data/Five_Stars_Hotels.csv',index=False)

===================================================================================================================================

4성급 호텔 웹크롤링

In [100]:
df.head()

,name,url,address,rating,review_rating,review_star,review
0,그랜드 워커힐 서울 (Grand Walkerhill Seoul),https://www.agoda.com/ko-kr/grand-walkerhill-s...,"[워커힐로 177, 광진, 서울, 대한민국, 04963]",8.8,9.12234,"[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....",[오래된 곳이러는 인식이 강해 어릴 적 외에 성인이 된 후 스스로 선택해 가 본 적...
1,스위스 그랜드 호텔 서울 (Swiss Grand Hotel Seoul),https://www.agoda.com/ko-kr/grand-hilton-seoul...,"[서대문구 연희로 353, 서대문, 서울, 대한민국, 03650]",8.0,9.9,"[10.0, 10.0, 10.0, 9.6, 9.6, 10.0, 10.0, 10.0,...",[동생이랑 서울여행 와서 2박 3일 동안 호텔에 머문게 젤 기억에 남을 정도로 편하...
2,메이필드 호텔 서울 (Mayfield Hotel Seoul),https://www.agoda.com/ko-kr/mayfield-hotel/hot...,None,8.6,None,None,None
3,워커힐 더글라스 하우스 (성인 전용) (Walkerhill Douglas House...,https://www.agoda.com/ko-kr/walkerhill-douglas...,None,9.1,None,None,None
4,롯데호텔 월드 (Lotte Hotel World),https://www.agoda.com/ko-kr/lotte-hotel-world/...,None,8.9,None,None,None


In [105]:
len(df.iloc[0]['review'])

752

In [102]:
len(df.iloc[1]['review_star'])

10